In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("../fin_health.csv")

In [3]:
X = df.drop(columns=['ID', 'Target'])
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc  = le.transform(y_test)

In [5]:

cat_features = X_train.select_dtypes(
    include=['object', 'category', 'bool']
).columns.tolist()

print("Categorical features:", cat_features)


Categorical features: ['country', 'attitude_stable_business_environment', 'attitude_worried_shutdown', 'compliance_income_tax', 'perception_insurance_doesnt_cover_losses', 'perception_cannot_afford_insurance', 'motor_vehicle_insurance', 'has_mobile_money', 'current_problem_cash_flow', 'has_cellphone', 'owner_sex', 'offers_credit_to_customers', 'attitude_satisfied_with_achievement', 'has_credit_card', 'keeps_financial_records', 'perception_insurance_companies_dont_insure_businesses_like_yours', 'perception_insurance_important', 'has_insurance', 'covid_essential_service', 'attitude_more_successful_next_year', 'problem_sourcing_money', 'marketing_word_of_mouth', 'has_loan_account', 'has_internet_banking', 'has_debit_card', 'future_risk_theft_stock', 'medical_insurance', 'funeral_insurance', 'motivation_make_more_money', 'uses_friends_family_savings', 'uses_informal_lender']


In [6]:


# Replace NaN with a string and ensure string dtype
for col in cat_features:
    X_train[col] = X_train[col].fillna('__MISSING__').astype(str)
    X_test[col]  = X_test[col].fillna('__MISSING__').astype(str)


In [7]:
cat_idx = [X.columns.get_loc(col) for col in cat_features]

# Convert cat_features to pd.Categorical dtype
for col in cat_features:
    X_train[col] = pd.Categorical(X_train[col])
    X_test[col] = pd.Categorical(X_test[col])

try this to check for the effect of categorical_feature=cat_idx, without using it i got --- Test performance ---
ROC AUC: 0.9432141147913701
Log loss: 0.30993043578069573
Accuracy: 0.8778586278586279

In [8]:
import pandas as pd
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score

# -------------------------------------------------
# 1. Hyperparameter search (NO early stopping here)
# -------------------------------------------------
param_dist = {
    'num_leaves': randint(31, 256),
    'max_depth': randint(4, 12),
    'learning_rate': uniform(0.01, 0.15),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_samples': randint(10, 100),
    'reg_alpha': uniform(0.0, 0.5),
    'reg_lambda': uniform(0.0, 0.5)
}

base_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    n_estimators=5000,   # large on purpose
    objective='multiclass',
    random_state=42,
    n_jobs=-1
)

search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='neg_log_loss',     # best for probabilistic multiclass
    refit=True,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

    
search.fit(X_train, y_train_enc, categorical_feature=cat_idx)

print("Best CV params:", search.best_params_)
print("Best CV score (neg log loss):", search.best_score_)

# -------------------------------------------------
# 2. Final model with EARLY STOPPING
# -------------------------------------------------
final_model = lgb.LGBMClassifier(
    **search.best_params_,
    boosting_type='gbdt',
    n_estimators=5000,   # early stopping will cap this
    objective='multiclass',
    random_state=42,
    n_jobs=-1
)

final_model.fit(
    X_train, y_train_enc,
    eval_set=[(X_test, y_test_enc)],
    eval_metric='multi_logloss',
    categorical_feature=cat_idx,
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(50)
    ]
)

print(f"\nBest iteration: {final_model.best_iteration_}")

# -------------------------------------------------
# 3. Final evaluation
# -------------------------------------------------
proba = final_model.predict_proba(X_test)
pred  = np.argmax(proba, axis=1)

print("\n--- Test performance ---")
print("ROC AUC:",
      roc_auc_score(y_test_enc, proba, multi_class='ovr', average='weighted'))
print("Log loss:", log_loss(y_test_enc, proba))
print("Accuracy:", accuracy_score(y_test_enc, pred))


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1048
[LightGBM] [Info] Number of data points in the train set: 6155, number of used features: 37
[LightGBM] [Info] Start training from score -3.017910
[LightGBM] [Info] Start training from score -0.426232
[LightGBM] [Info] Start training from score -1.210220
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7f577150a680>
Traceback (most recent call last):
  File "/home/ismail/miniconda/envs/ml-dl/lib/python3.10/site-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function _log_callback at 0x7f25d2fb2680>
Traceback (most recent call last):
  File "/home/ismail/miniconda/envs/ml-dl/lib/python3.10/site-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function _log_callback at 0x7f4a56b4a680>
Traceback (most recent call last):
  File "/home/ismail/miniconda/envs/ml-dl/lib/python3.10/site-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <fu

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

KeyboardInterrupt: 

In [8]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score
from scipy.stats import randint, uniform
import time


#param_dist = {
#    'depth': randint(4, 10),
#    'learning_rate': uniform(0.01, 0.15),
#    'l2_leaf_reg': uniform(1.0, 9.0),
#    'bagging_temperature': uniform(0.0, 1.0),
#}

param_dist = {
    'depth': randint(5, 9),
    'learning_rate': uniform(0.03, 0.1),
    'l2_leaf_reg': uniform(2.0, 6.0),
    'bagging_temperature': uniform(0.0, 0.7)
    
}

base_model = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='MultiClass',
    iterations=800,
    early_stopping_rounds=50,    # ✅ STOP BAD MODELS
    random_seed=42,
    cat_features=cat_features,
    verbose=0
)

search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring='neg_log_loss',
    refit=True,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

start = time.time()
search.fit(X_train, y_train_enc)
end = time.time()

print("Best params:", search.best_params_)
print("Best CV score (neg log loss):", search.best_score_)
print(f"Search time: {end - start:.2f} seconds")




Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best params: {'bagging_temperature': 0.5658781436815228, 'depth': 5, 'l2_leaf_reg': 2.095797513321285, 'learning_rate': 0.053089382562214904}
Best CV score (neg log loss): -0.31063124952755955
Search time: 1287.80 seconds


In [9]:
num_classes = len(np.unique(y_train_enc))

final_model = CatBoostClassifier(
    **search.best_params_,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    iterations=5000,
    random_seed=42,
    cat_features=cat_features,
    verbose=50,
    od_type='Iter',
    od_wait=100
)

start = time.time()
final_model.fit(
    X_train, y_train_enc,
    eval_set=(X_test, y_test_enc),
    use_best_model=True
)
end = time.time()

print(f"\nBest iteration: {final_model.get_best_iteration()}")
print(f"Training time: {end - start:.2f} seconds")


0:	learn: 1.0433335	test: 1.0439402	best: 1.0439402 (0)	total: 14ms	remaining: 1m 10s
50:	learn: 0.3882167	test: 0.3951008	best: 0.3951008 (50)	total: 731ms	remaining: 1m 10s
100:	learn: 0.3259079	test: 0.3367241	best: 0.3367241 (100)	total: 1.47s	remaining: 1m 11s
150:	learn: 0.3121247	test: 0.3250059	best: 0.3250059 (150)	total: 2.2s	remaining: 1m 10s
200:	learn: 0.3035733	test: 0.3198475	best: 0.3198475 (200)	total: 2.94s	remaining: 1m 10s
250:	learn: 0.2974071	test: 0.3165250	best: 0.3165250 (250)	total: 3.68s	remaining: 1m 9s
300:	learn: 0.2919034	test: 0.3143257	best: 0.3143257 (300)	total: 4.39s	remaining: 1m 8s
350:	learn: 0.2875650	test: 0.3126246	best: 0.3126129 (346)	total: 5.13s	remaining: 1m 8s
400:	learn: 0.2829771	test: 0.3117743	best: 0.3117743 (400)	total: 5.89s	remaining: 1m 7s
450:	learn: 0.2785417	test: 0.3107034	best: 0.3107034 (450)	total: 6.68s	remaining: 1m 7s
500:	learn: 0.2738285	test: 0.3097478	best: 0.3097471 (497)	total: 7.43s	remaining: 1m 6s
550:	learn: 0

In [10]:
proba = final_model.predict_proba(X_test)
pred  = np.argmax(proba, axis=1)

print("\n--- CatBoost Test Performance ---")
print(f"ROC AUC (ovr weighted): {roc_auc_score(y_test_enc, proba, multi_class='ovr', average='weighted'):.5f}")
print(f"Log Loss: {log_loss(y_test_enc, proba):.5f}")
print(f"Accuracy: {accuracy_score(y_test_enc, pred):.5f}")



--- CatBoost Test Performance ---
ROC AUC (ovr weighted): 0.94567
Log Loss: 0.30498
Accuracy: 0.87734


🚀 Optional improvement (recommended for imbalance)

Add this to your final model:
```
auto_class_weights='Balanced'
```

Example:
```
final_model = CatBoostClassifier(
    **search.best_params_,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    iterations=5000,
    cat_features=cat_features,
    auto_class_weights='Balanced',
    random_seed=42,
    verbose=50,
    od_type='Iter',
    od_wait=100
)
```